In [ ]:
# importing Libraries

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing and reading dataset

ds= pd.read_csv("new_cleaned_data.csv")
ds

In [ ]:
X=ds[['company','name','year','kms_driven','fuel_type']]
y=ds[['Price']]

In [ ]:
# Implementing OneHotcoder

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe=OneHotEncoder()
ohe.fit(X[["company","name","fuel_type"]])

In [ ]:
ct=make_column_transformer((OneHotEncoder(handle_unknown = 'ignore',categories=ohe.categories_), ["company","name","fuel_type"]),remainder='passthrough',force_int_remainder_cols=False, sparse_threshold = 0)
ct

In [ ]:
# making pipeline

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

reg=LinearRegression()
regDec = DecisionTreeRegressor(random_state = 0)
regRFR = RandomForestRegressor(n_estimators =10, random_state= 0)

pipeLinear = make_pipeline(ct, reg)
pipeDec = make_pipeline(ct, regDec)
pipeRFR = make_pipeline(ct, regRFR)

scores = []
for i in range(0,101):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state=i)
    pipeLinear.fit(X_train,y_train)
    pipeDec.fit(X_train,y_train)
    pipeRFR.fit(X_train,y_train)

    result = pipeLinear.predict(X_test)
    scoreLinear = r2_score(y_test, result)
    rmseLinear = np.sqrt(mean_squared_error(y_test, result))
    
    result = pipeDec.predict(X_test)
    scoreDec = r2_score(y_test, result)
    rmseDec = np.sqrt(mean_squared_error(y_test, result))
    result = pipeRFR.predict(X_test)
    scoreRFR = r2_score(y_test, result)
    rmseRFR = np.sqrt(mean_squared_error(y_test, result))
    
    
    scores.append(('Linear',i, scoreLinear,rmseLinear))
    scores.append(('Decision',i, scoreDec, rmseDec ))
    scores.append(('Random Forest',i, scoreRFR, rmseRFR ))

In [ ]:
# Finding best r2 score and RMSE score

scoreDF = pd.DataFrame(data = scores, columns = ["algo", "Iteration", "R2 score", "RMSE Score"])
resultDF = scoreDF.sort_values(by="R2 score", ascending = False)
resultDF
# r2 score must be maximum and RMSE score must be minimum

In [ ]:
# Making Pipeline for best suited algorithm(Decision tree) 

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state=79)
pipeDec.fit(X_train,y_train)

In [ ]:
# predict for my_input

company=input("enter company name : ")
name=input("enter car name : ")
year=int(input("enter year : "))
kms_driven=int(input("enter kms driven : "))
fuel_type=input("enter fuel type : ")
columns=["company","name","year","kms_driven","fuel_type"]
myinput=pd.DataFrame(columns=columns,data=[[company,name,year,kms_driven,fuel_type]])
result=pipeDec.predict(myinput)
print("You should buy it for ~ price : ",result)

In [ ]:
# Importing to pkl

import pickle as pkl
pkl.dump(pipeRFR, open("CarProjectUsing_other_algos.pkl","wb"))